[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/see-mof/ssdp/main?filepath=lectures%2F6%2Fssdp_lecture_6_exercises.ipynb)

# A diagram application

In [ ]:
!pip install ipycanvas
!pip install numpy
from ipycanvas import Canvas, hold_canvas

# Exercise 1: A 2D coordinates class

Implement the `Coordinates` shown in the UML class so that the assertions in the next cell are satisfied.

## UML class diagram:

![Coordinates class UML diagram](https://raw.githubusercontent.com/SEE-MOF/ssdp/main/lectures/6/figures/diagram_coordinates.svg)

In [ ]:
class Coordinates:
    pass

In [ ]:
coords_1 = Coordinates(1, 2)
coords_2 = Coordinates(2, 3)
coords_3 = coords_1 + coords_2
coords_4 = coords_1 * 0.5

assert coords_3.x == 3
assert coords_3.y == 5
assert coords_4.x == 0.5
assert not coords_1 == coords_2
assert str(coords_1) == "Coordinates(1, 2)"

# Exercise 2: Inheritance

Below you find definitions for most classes presented in the lecture. Read through
them so you understand how to use them. Then complete the definition of the ``Node``
class so that according to the UML class diagram below and the assert statements in
last cell.

## UML Class diagram


<img src="https://raw.githubusercontent.com/SEE-MOF/ssdp/main/lectures/6/figures/diagram_inheritance_2.svg" alt="Inheritance class diagram." style="width:600px;"/>

In [ ]:
class Color:
    """
    The color of diagram components.
    
    Attributes:
        color_code(``str``): The color represented in HTML HEX string format.
    """
    @staticmethod
    def Black():
        """The color black."""
        return Color("#000000")
    
    @staticmethod
    def Red():
        """The color red."""
        return Color("#FF0000")
    
    @staticmethod
    def Green():
        """The color green."""
        return Color("#00FF00")
    
    @staticmethod
    def Blue():
        """The color blue."""
        return Color("#0000FF")
        
    def __init__(self, color_code):
        """ Create color from given color code. """
        self.color_code = color_code
        
    def __str__(self):
        return self.color_code

In [ ]:
class DiagramComponent:
    """
    Base class for diagrams component.
    
    Attributes:
        position(Coordinates): The component's position represented as
            as Coordinates object.
        color(Color): The components color represented as Color
            as Color object.
    """
    def __init__(self, position, color):
        """
        Create diagram component.
        
        Args:
            position(Coordinates): The position of the object
            color(Color): The color of the object.
        """
        self.position = position
        self.color = color
        
    def translate(self, delta):
        """
        Translate object by given direction.
        
        Args:
            delta(Coordinates): Coordinates object representing the direction
                step by which to translate the object.
        """
        self.position = self.position + delta
        
    def set_color(self, new_color):
        """
        Set color of component.
        
        Args:
            new_color(Color): The new color of the component.
        """
        self.color = new_color

In [ ]:
class Rectangle(DiagramComponent):
    """
    A filled rectangle.
    
    Attributes:
        dimensions(Coordinates): Coordinates object holding the width
            and height of the rectangle.
    """
    def __init__(self,
                 position,
                 dimensions,
                 color = Color.Red()):
        """
        Create rectangle.
        
            Args:
                position(Coordinates): The position of the upper left corner of the
                    rectangle.
                dimensions(Coordinates): Coordinates object holding the horizontal
                    and vertical extent of the rectangle
                color(Color): The color with which to fill rectangle
        """
        super().__init__(position, color)
        self.dimensions = dimensions
        
    def draw(self, canvas):
        """
        Draw rectangle on canvas.
        
        Uses the ``ipycanvas`` API to draw a filled rectangle on the given HTML5 canvas
        object.
        
        Args:
            canvas(ipycanvas.Canvas): Canvas to draw the rectangle on.
        """
        canvas.fill_style = str(self.color)
        canvas.fill_rect(self.position.x,
                         self.position.y,
                         self.dimensions.x,
                         self.dimensions.y)
        
    @property
    def left(self):
        return self.position + Coordinates(0, self.dimensions.y / 2)
    
    @property
    def top_left(self):
        return self.position
    
    @property
    def top(self):
        return self.position + Coordinates(self.dimensions.x / 2, 0)
    
    @property
    def top_right(self):
        return self.position + Coordinates(self.dimensions.x, 0)
    
    @property
    def right(self):
        return self.position + Coordinates(self.dimensions.x, self.dimensions.y / 2)
    
    @property
    def bottom_right(self):
        return self.position + Coordinates(self.dimensions.x, self.dimensions.y)
    
    @property
    def bottom(self):
        return self.position + Coordinates(self.dimensions.x / 2, self.dimensions.y)
    
    @property
    def bottom_left(self):
        return self.position + Coordinates(0, self.dimensions.y)

In [ ]:
class Text(DiagramComponent):
    """
    A colored text in a diagram.
    """
    def __init__(self,
                 text,
                 position,
                 color=Color.Black()):
        """
        Create text object.
        
        Args:
            text(str): The text
            position(Coordinates): Position around which to center the text.
            color(Color): The fill color to use for the text.
        """
        super().__init__(position, color)
        self.text = text
        
    def draw(self, canvas):
        """
        Draw text on canvas.
        
        Uses the ``ipycanvas`` API to draw  filled text on the given HTML5 canvas
        object.
        
        Args:
            canvas(ipycanvas.Canvas): Canvas to draw the rectangle on.
        """
        canvas.font = "16pt sans"
        canvas.text_align = "center"
        canvas.fill_style = str(self.color)
        canvas.fill_text(self.text, self.position.x, self.position.y)

In [ ]:
import numpy as np

class Arrow(DiagramComponent):
    """
    A arrow in a diagram.
    
    Attributes:
        end(Coorinates): End position of the arrow.
    """
    def __init__(self, start, end, color=Color.Black(), head_size=10):
        """
        Create arrow.
        
        Args:
            start(Coordinates): Start position of arrow.
            end(Coordinates): End position of arrow.
            color(Color): Arrow color.
            head_size(int): Size of arrow head in pixels.
        """
        super().__init__(start, color)
        self.end = end
        self.head_size = head_size
        
    def draw(self, canvas):
        """
        Draw arrow on canvas.
        
        Uses the ``ipycanvas`` API to draw  arrow text on the given HTML5 canvas
        object.
        
        Args:
            canvas(ipycanvas.Canvas): Canvas to draw the rectangle on.
        """
        canvas.stroke_style = str(self.color)
        canvas.begin_path()
        canvas.move_to(self.position.x, self.position.y)
        canvas.line_to(self.end.x, self.end.y)
        canvas.stroke()
        canvas.close_path()    
        
        angle = np.pi + np.arctan2(self.end.y - self.position.y,
                                   self.end.x - self.position.x)
        x_1 = self.end.x + self.head_size * np.cos(angle + np.pi / 6)
        y_1 = self.end.y + self.head_size * np.sin(angle + np.pi / 6)
        x_2 = self.end.x + self.head_size * np.cos(angle - np.pi / 6)
        y_2 = self.end.y + self.head_size * np.sin(angle - np.pi / 6)
        canvas.begin_path()
        canvas.move_to(self.end.x, self.end.y)
        canvas.line_to(x_1, y_1)
        canvas.line_to(x_2, y_2)
        canvas.line_to(self.end.x, self.end.y)
        canvas.fill()
        canvas.close_path()

In [ ]:
class Node(DiagramComponent):
    ...

In [ ]:
node = Node(Coordinates(200, 200),
            Coordinates(100, 100),
            "node")

assert isinstance(node, DiagramComponent)
assert node.text.text == "node"
assert node.rectangle.position == Coordinates(200, 200)
assert node.rectangle.dimensions == Coordinates(100, 100)

# Exercise 3: Using the DiagramComponent interface

Complete the definition of the ``Diagram`` class below, so that the code in the last cell works.

In [ ]:
from ipycanvas import Canvas, hold_canvas
class Diagram:
    def __init__(self, width, height):
        ...
        
        ...
        
    def draw(self):
        canvas = Canvas(width=self.width, height=self.height)
        with hold_canvas(canvas):
            ...
        return canvas

In [ ]:
diagram = Diagram(400, 200)
node_1 = Node(Coordinates(50, 50),
              Coordinates(100, 100), "node 1")
node_2 = Node(Coordinates(250, 50),
              Coordinates(100, 100), "node 2")
node_2.set_color(Color.Blue())
arrow = Arrow(node_1.rectangle.right, node_2.rectangle.left)

diagram.add(node_1)
diagram.add(node_2)
diagram.add(arrow)
canvas = diagram.draw()
canvas